In [1]:
!pip install langchain langchain-core langchain-community pydantic duckduckgo-search langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.6 MB/s eta 0:00:00


## Built-in Tool - DuckDuckGo Search

In [1]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

results = search_tool.invoke('Founder of Jainism')


print(results)

f:\GEN-AI NOTES\LangChain\LC\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Aug 22, 2025 · Learn about the origin and development of Jainism, a sramanic religion that emerged in India in the 6th century BC. The founder of Jainism was Vardhamana Mahavira, the … Jul 29, 2025 · Scholars of religion generally hold that Jainism originated in the 7th–5th century bce in the Ganges basin of eastern India, the scene of intense religious speculation and activity at … Aug 6, 2025 · Just like Buddha didn’t “invent” Dhamma but rediscovered it, Mahavira didn’t create Jainism. Instead, he was the 24th Tirthankara — a title for someone who has attained perfect … May 21, 2025 · Mahavira, the 24th Tirthankara, is considered the first real founder of Jainism for formalizing its teachings and organizing its community around 599–527 BCE. His emphasis on … May 1, 2025 · Jainism is an ancient religion that originated in India. It traces its history through twenty-four tirthankaras (spiritual teachers), with Rishabhdev being the first tirthankara. Jainism …


In [2]:
print(search_tool.name)
print(search_tool.description)
print(search_tool.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


## Built-in Tool - Shell Tool

In [8]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()

results = shell_tool.invoke('whoami')

print(results)

Executing command:
 whoami
root



/usr/local/lib/python3.11/dist-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


## Custom Tools

In [9]:
from langchain_core.tools import tool

In [10]:
# Step 1 - create a function with docstring

def multiply(a, b):
    """Multiply two numbers"""
    return a*b

In [13]:
# Step 2 - add type hints

def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [14]:
# Step 3 - add tool decorator

@tool
def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [15]:
result = multiply.invoke({"a":3, "b":5})

In [16]:
print(result)

15


In [17]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [18]:
print(multiply.args_schema.model_json_schema())

{'description': 'Multiply two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


## Method 2 - Using StructuredTool

In [19]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

In [20]:
class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

In [21]:
def multiply_func(a: int, b: int) -> int:
    return a * b

In [22]:
multiply_tool = StructuredTool.from_function(
    func=multiply_func,
    name="multiply",
    description="Multiply two numbers",
    args_schema=MultiplyInput
)

In [23]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


## Method 3 - Using BaseTool Class

In [24]:
from langchain.tools import BaseTool
from typing import Type

In [25]:
# arg schema using pydantic

class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

In [26]:
class MultiplyTool(BaseTool):
    name: str = "multiply"
    description: str = "Multiply two numbers"

    args_schema: Type[BaseModel] = MultiplyInput

    def _run(self, a: int, b: int) -> int:
        return a * b

In [27]:
multiply_tool = MultiplyTool()

In [28]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)

print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


## Toolkit


<span style="color:red">Collections of tools for ease of access</span>





In [29]:
from langchain_core.tools import tool

# Custom tools
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b


In [30]:
class MathToolkit:
    def get_tools(self):
        return [add, multiply]


In [31]:
toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool in tools:
    print(tool.name, "=>", tool.description)


add => Add two numbers
multiply => Multiply two numbers


In [33]:
result = tools[0].invoke({"a": 2, "b": 3})
print(result)

5


In [35]:
tools[0]

StructuredTool(name='add', description='Add two numbers', args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x79b1b238d440>)